In [ ]:
import json
import pandas as pd
import numpy as np

### Utils

In [ ]:
n_runs = 5
def average_over_runs(basename):    
    p50_00_latency = []
    p99_00_latency = []
    p99_90_latency = []
    for i in range(n_runs):
        curname = basename + '_' + str(i)
        with open(curname, 'r') as file:
            result =  json.load(file)
            p50_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p50.00'])
            p99_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.00'])
            p99_90_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.90'])
    result_dict = {}
    #result_dict["sender_type"] = sender[7:]
    #result_dict["payload_size"] = size
    result_dict["simult_req"] = n_runs
    #result_dict["payload_bound"] = payload_bound
    #result_dict["circle_size"] = circle_size

    result_dict["p50.00"] = np.mean(p50_00_latency)
    result_dict["p99.00"] = np.mean(p99_00_latency)
    result_dict["p99.90"] = np.mean(p99_90_latency)
    
    result_dict["std_p50.00"] = np.std(p50_00_latency)
    result_dict["std_p99.00"] = np.std(p99_00_latency)
    result_dict["std_p99.90"] = np.std(p99_90_latency)

    return result_dict



### Get results

In [ ]:
sender_type = ['sender_rdma_write_nomultip_read']
payload_sizes = ['10-100', '100-1000', '1000-10000']
simult_req = ['1', '10', '100']
payload_bounds = ['1500']
circle_sizes = ['200', '500', '1000', '5000', '10000']

data = []

for size in payload_sizes:
   for n in simult_req:
      for payload_bound in payload_bounds:
         for circle_size in circle_sizes:
            for sender in sender_type:
               basename = f"results/json_{sender}_{size}_{n}_{payload_bound}_{circle_size}"
               result_dic = average_over_runs(basename)
               result_dic["sender_type"] = sender[7:]
               result_dic["payload_size"] = size
               result_dic["simult_req"] = n
               result_dic["payload_bound"] = payload_bound
               result_dic["circle_size"] = circle_size
               data.append(result_dic)

df = pd.DataFrame(data)
df.to_csv(f'data/res_vary_circle.csv')


In [ ]:
sender_type = ['sender_rdma_write_nomultip_read']
payload_sizes = ['10-100', '100-1000', '1000-10000']
simult_req = ['1', '10', '100']
payload_bounds = ['100', '500', '1500', '3000', '10000']
circle_sizes = ['200']

data = []

for size in payload_sizes:
   for n in simult_req:
      for payload_bound in payload_bounds:
         for circle_size in circle_sizes:
            for sender in sender_type:
               basename = f"results/json_{sender}_{size}_{n}_{payload_bound}_{circle_size}"
               result_dic = average_over_runs(basename)
               result_dic["sender_type"] = sender[7:]
               result_dic["payload_size"] = size
               result_dic["simult_req"] = n
               result_dic["payload_bound"] = payload_bound
               result_dic["circle_size"] = circle_size
               data.append(result_dic)

df = pd.DataFrame(data)
df.to_csv(f'data/res_vary_payloadBound.csv')


### agg results in plots

In [ ]:
# Build (sender_type, payload_size) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_vary_circle.csv')

simult_req = [1, 10, 100]
payload_bounds = [1500]
circle_sizes = [200, 500, 1000, 5000, 10000]
metrics = ['p50.00', 'p99.00', 'p99.90']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for n in simult_req:
    for payload_bound in payload_bounds:
        for metric in metrics:
            plt.clf() # Reset plot
            df_fixed = df[(df['simult_req'] == n) & (df['payload_bound'] == payload_bound)]

            df_comp_fixed_temp = df_fixed.pivot("circle_size", "payload_size", metric)
            yerr = df_fixed.pivot("circle_size", "payload_size", f"std_{metric}")
            #print("yerr \n", yerr, "end")
            #print("df_cimp_fixed_temp \n", df_comp_fixed_temp, "end")
            ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

            # Set plot title and labels
            plt.title(f"{metric} Latency comparison \n for simult_req={n}, payload_bound={payload_bound}")
            plt.xlabel('Prototype')
            plt.ylabel(f"{metric} Latency (ms)")

            # Orient sender_type labels at 45 degrees
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.tick_params(axis='x', pad=10)

            # Adjust plot layout
            plt.legend(title='Data Size', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            # Show or save the plot
            plt.savefig(f'plots/vary-circle_size-sender-comp-payload_size-{n}_{payload_bound}_{metric}.svg', bbox_inches='tight', format='svg')
            #plt.show()

In [ ]:
# Build (sender_type, simult_req) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_vary_circle.csv')

payload_sizes = ['10-100','100-1000','1000-10000']
payload_bounds = [1500]
circle_sizes = [200, 500, 1000, 5000, 10000]
metrics = ['p99.90', 'p99.00', 'p50.00']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for size in payload_sizes:
    for payload_bound in payload_bounds:
        for metric in metrics:
            plt.clf() # Reset plot
            df_fixed = df[(df['payload_size'] == size) & (df['payload_bound'] == payload_bound)]
            df_comp_fixed_temp = df_fixed.pivot("circle_size", "simult_req", metric)
            yerr = df_fixed.pivot("circle_size", "simult_req", f"std_{metric}")
            ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

            # Set plot title and labels
            plt.title(f"{metric} Latency comparison \n for data_size={size}, payload_bound={payload_bound}")
            plt.xlabel('Prototype')
            plt.ylabel(f"{metric} Latency (ms)")

            # Orient sender_type labels at 45 degrees
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.tick_params(axis='x', pad=10)

            # Adjust plot layout
            plt.legend(title='Simultaneous Requests', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            # Save the plot
            #plt.show()
            plt.savefig(f'plots/vary-circle_size-sender-comp-simult_req-{size}_{payload_bound}_{metric}.svg', bbox_inches='tight', format='svg')

### same for payloadBound

In [ ]:
# Build (sender_type, payload_size) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_vary_payloadBound.csv')

simult_req = [1, 10, 100]
payload_bounds = [100, 500, 1500, 3000, 10000]
circle_sizes = [200]
metrics = ['p50.00', 'p99.00', 'p99.90']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for n in simult_req:
    for circle_size in circle_sizes:
        for metric in metrics:
            plt.clf() # Reset plot
            df_fixed = df[(df['simult_req'] == n) & (df['circle_size'] == circle_size)]

            df_comp_fixed_temp = df_fixed.pivot("payload_bound", "payload_size", metric)
            yerr = df_fixed.pivot("payload_bound", "payload_size", f"std_{metric}")
            #print("yerr \n", yerr, "end")
            #print("df_cimp_fixed_temp \n", df_comp_fixed_temp, "end")
            ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

            # Set plot title and labels
            plt.title(f"{metric} Latency comparison \n for simult_req={n}, circle_size={circle_size}")
            plt.xlabel('Prototype')
            plt.ylabel(f"{metric} Latency (ms)")

            # Orient sender_type labels at 45 degrees
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.tick_params(axis='x', pad=10)

            # Adjust plot layout
            plt.legend(title='Data Size', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            # Show or save the plot
            plt.savefig(f'plots/vary-payload_bound-sender-comp-payload_size-{n}_{payload_bound}_{metric}.svg', bbox_inches='tight', format='svg')
            #plt.show()

In [ ]:
# Build (sender_type, simult_req) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_vary_payloadBound.csv')

payload_sizes = ['10-100','100-1000','1000-10000']
payload_bounds = [100, 500, 1500, 3000, 10000]
circle_sizes = [200]
metrics = ['p99.90', 'p99.00', 'p50.00']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for size in payload_sizes:
    for circle_size in circle_sizes:
        for metric in metrics:
            plt.clf() # Reset plot
            df_fixed = df[(df['payload_size'] == size) & (df['circle_size'] == circle_size)]
            df_comp_fixed_temp = df_fixed.pivot("payload_bound", "simult_req", metric)
            yerr = df_fixed.pivot("payload_bound", "simult_req", f"std_{metric}")
            ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

            # Set plot title and labels
            plt.title(f"{metric} Latency comparison \n for data_size={size}, circle_size={circle_size}")
            plt.xlabel('Prototype')
            plt.ylabel(f"{metric} Latency (ms)")

            # Orient sender_type labels at 45 degrees
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.tick_params(axis='x', pad=10)

            # Adjust plot layout
            plt.legend(title='Simultaneous Requests', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            # Save the plot
            #plt.show()
            plt.savefig(f'plots/vary-payload_bound-sender-comp-simult_req-{size}_{payload_bound}_{metric}.svg', bbox_inches='tight', format='svg')